In [1]:
# If using Google colab, mount the Google drive
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import re
import os
import sys
import string
import numpy as np
import pandas as pd
import nltk
import json
import copy
from typing import Union
from nltk.tokenize import TweetTokenizer
nltk.download('punkt')

ModuleNotFoundError: No module named 'pandas'

In [2]:
import tensorflow as tf

2022-10-31 15:15:16.444133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)

from tensorflow.keras import Sequential, Model, constraints
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM, Bidirectional, Masking, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow'

### **Load necessary files**

In [9]:
root_path = os.path.abspath(os.getcwd())

'/Users/zzkzz/Documents/CS5344/Project/CS5344_Influencer_Cluster_Sentiment_Analysis/sentiment_analysis'

In [5]:
# Load necessary files

with open(os.path.join(root_path, 'data/english_contractions.json'), 'r') as f1:
    eng_contractions = json.load(f1)
f1.close()

with open(os.path.join(root_path, 'data/vocab_to_idx.json'), 'r') as f2:
    word_to_idx_dict = json.load(f2)

### **Define key parameters**

In [11]:
seqLen = 15
minLen = 8
neutral_threshold = 0.8
tokenizer = TweetTokenizer()

# Recording the relationship between idx and class
class_idx_dict = {'neutral':-1, 'happiness':0, 'fun':1, 'sadness':2, 'hate':3} # Since `neutral` label is not directly included in our prediction, we mark it as `-1`
idx_class_dict = {idx:class_label for class_label, idx in class_idx_dict.items()}

num_classes = len(class_idx_dict) - 1
vocab_size = len(word_to_idx_dict) + 1

In [12]:
# Sample sentence
# sentence = 'I feel bad today. I think I should go to see a doctor. @father'

### **Define necessary preprocessing functions**

In [10]:
def normalize_contractions(sentence, eng_contractions_dict):
    return _normalize_contractions_text(sentence, eng_contractions_dict)

def _normalize_contractions_text(text, contractions):
    """
    This function normalizes english contractions (all input sentences in lower case).
    """
    new_token_list = []
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        if word in contractions:
            replacement = contractions[word]

            first_rep = replacement.strip().split('/')[0]
            replacement_tokens = first_rep.strip().split()
            for w in replacement_tokens:
                new_token_list.append(w)
        else:
            new_token_list.append(word)
    sentence = " ".join(new_token_list).strip(" ")
    return sentence

def simplify_punctuation_and_whitespace(sentence):

    # print("Normalizing whitespaces and punctuation")
    sent = _replace_urls(sentence)
    sent = _simplify_punctuation(sent)
    simplified_sent = _normalize_whitespace(sent)
      
    return simplified_sent

def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "<URL>", text)
    return text

def _simplify_punctuation(text):
    """
    This function simplifies doubled or more complex punctuation. The exception is '...'.
    """
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1', corrected)
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

def _normalize_whitespace(text):
    """
    This function normalizes whitespaces, removing duplicates.
    """
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

def reduce_exaggerations(text):
    """
    Auxiliary function to help with exxagerated words.
    Examples:
        woooooords -> words
        yaaaaaaaaaaaaaaay -> yay
    """
    correction = str(text)
    #TODO work on complexity reduction.
    return re.sub(r'(.)\1+', r'\1\1', correction)

def sentence_tokenizer(tk, sentence, word_to_idx_dict):
    words_list = tk.tokenize(sentence)
    tokenized_words_list = []
    i = 0

    while i < len(words_list):
        if words_list[i].startswith('http') or words_list[i].startswith('www.'):
            i += 1
        elif words_list[i].endswith('.com'):
            i += 1
        # elif words_list[i-1] in string.punctuation and words_list[i] in string.punctuation:
        #     i += 1
        elif words_list[i] in string.punctuation:
            i += 1
        elif (len(words_list[i]) > 1) and (not (ord('a') <= ord(words_list[i][0]) <= ord('z'))) and (not words_list[i].startswith('<')):
            i += 1
        elif words_list[i].startswith('@'):
            tokenized_words_list.append('<person>')
            i += 1
        else:
            tokenized_words_list.append(words_list[i])
            i += 1

    for j, w in enumerate(tokenized_words_list):
        if w not in word_to_idx_dict:
            tokenized_words_list[j] = 'unk'

    return tokenized_words_list


# Main

def preprocess_user_sentence(sentence, seqLen, eng_contractions, 
                             word_to_idx_dict, 
                             tokenizer):
    sentence = sentence.lower()
    s = normalize_contractions(sentence, eng_contractions)
    s = simplify_punctuation_and_whitespace(s)
    s = reduce_exaggerations(s)
    tokens = sentence_tokenizer(tokenizer, sentence, word_to_idx_dict)
    tokens_in_numbers = [word_to_idx_dict[w] for w in tokens]
    
    return tokens_in_numbers

In [14]:
def tweet_preprocess(inputs: Union[pd.DataFrame, pd.Series], minLen=8, tokenizer=TweetTokenizer(), sentence_tokenize=False):
    """
    Function to preprocess tweets
    inputs: A dataframe in the format of |id|tweet1|tweet2|...|tweetn|, |id|tweet|, or |tweet|
    outputs: A dataframe in the format of |id|tweet_index|tweet|tokens|most_common_sentiment|
    sentence_tokenize: Determine if need to split text into sentences
    """

    data = inputs.copy(deep=True)

    if data.shape[1]>2: # If there are various tweety in one row, transform them into the format of |id|tweet|
        data.columns = ['id'] + ['tweets_'+str(i) for i in range(1, data.shape[1])]
        data = data.set_index(['id'])
        data = data.stack()
        data = data.reset_index()
        data.columns = ['id', 'tweet_index', 'tweet']
        # data = data.drop(['tweet_index'], axis=1)
        # data.dropna()

    elif data.shape[1]==2: # Data is in the format of |id|tweet|
        data.columns = ['id', 'tweet']

    else: # Data is in the format of |tweet|
        data.columns = ['tweet']
    
    if sentence_tokenize: # Determine whether to split text into sentences
        data = sentences_tokenizer(data)
    
    # Standize some characters
    data['tweet'] = data['tweet'].map(lambda tweet: tweet.replace('¡¯', '\'').replace('¡', '...'))

    # Tokenize text
    data['tokens'] = data['tweet'].map(lambda tweet: preprocess_user_sentence(tweet, seqLen, eng_contractions, word_to_idx_dict, tokenizer)[0])
    data = data[data['tokens'].map(lambda x: len(x) > minLen)]

    padded_sentence = pad_sequences(data['tokens'], maxlen=seqLen, padding='post', truncating='pre')
    data['tokens'] = [list(doc) for doc in padded_sentence]

    return data


def tweet_sentiment_predict(tweet_df, threshold, trained_model, save_file=False):
    """
    Function to predict tweet sentiment for different users
    threshold: When the highest score is lower than threshold, predicted label is set as `neutral`
    outputs: A dataframe in the format of |id|neutral|happiness|...|most_common_sentiment|; A dictionary containing ids and corresponding sentiments
    """
    # Predict
    type_scores = trained_model.predict(np.asarray(tweet_df['tokens'].values.tolist()))

    # Decode sentiment
    highest_scores = [np.max(score, axis=-1) for score in type_scores]
    idx_highest_scores = [np.argmax(score, axis=-1) for score in type_scores]
    labels = [idx_class_dict[idx] if score>=threshold else idx_class_dict[-1] for idx, score in zip(idx_highest_scores, highest_scores)]
    tweet_df["tag"] = pd.DataFrame(labels)

    # Drop useless column
    # if 'tweet_index' in tweet_df.columns:
    #     tweet_df = tweet_df.drop(['token'], axis=1)
    # if 'tweet' in tweet_df.columns:
    #     tweet_df = tweet_df.drop(['sentence'], axis=1)

    # Count # of each type
    tweet_df = tweet_df.groupby(['id', 'tag'])['tag'].count()
    tweet_df = tweet_df.unstack()
    tweet_df = tweet_df.fillna(0)
    tweet_df = tweet_df.reset_index()
    tweet_df.iloc[:,1:] = tweet_df.iloc[:,1:].astype('int64')
    tweet_df.iloc[:,1:] = tweet_df.iloc[:,1:].div(tweet_df.iloc[:,1:].sum(axis=1), axis=0).round(4)
    
    # Find the most common type
    sentiment_df = tweet_df.iloc[:,1:]
    tweet_df['most_common_sentiment'] = sentiment_df.idxmax(axis=1)

    sentiment_dict = {}
    ids, sentiments = tweet_df['id'].values.tolist(), tweet_df['most_common_sentiment'].values.tolist()
    for i in range(len(ids)):
        sentiment_dict[ids[i]] = sentiments[i]

    if save_file:
        tweet_df.to_csv(os.path.join(root_path, 'data/sentiment.txt'), sep=',', index=False)

    return tweet_df, sentiment_dict

In [15]:
def postprocessing(id_df, sentiment_df, id_to_sentiment_dict):
    all_ids = pd.unique(id_df['id'])
    has_sentiment_label_ids = set(pd.unique(sentiment_df['id']))

    # Add labels for those ids without valid sentences
    for id in all_ids:
        if id not in has_sentiment_label_ids:
            id_to_sentiment_dict[id] = 'neutral'

    # Combine id_df and sentiment_df to get final result
    sentiment_predicted_results = pd.merge(id_df, sentiment_df, on=['id']).sort_values(['center', 'id'])
    
    return sentiment_predicted_results, id_to_sentiment_dict

In [16]:
# Optional
from nltk.tokenize import sent_tokenize

def sentences_tokenizer(inputs: pd.DataFrame):
    """
    Auxiliary function to split text into sentences.
    inputs: A dataframe in the format of |user_id|tweet|
    outputs: A dataframe in the format of |user_id|sentences|
    Examples:
        I feel bad today. I think I should go to see a doctor. -> I feel bad today.
                                            I think I should go to see a doctor.
    """
    outputs = inputs.copy(deep=True)
    outputs["sentences"] = outputs["tweet"].map(lambda tweet: sent_tokenize(tweet))
    outputs = outputs.drop(["tweet"], axis=1)
    outputs = outputs.explode("sentences")
    outputs.columns = ['id', "sentence"]
    outputs = outputs.dropna()

    outputs = outputs.reset_index()
    outputs = outputs.drop(["index"], axis=1)
    return outputs

### **Option A: Load the whole model**

In [ ]:
# Load model
model_path = os.path.join(root_path, 'lstm_model_v1')
# print(model_path)
trained_model = load_model(model_path)

In [ ]:
# Test
# tk_test = TweetTokenizer()
# tokens = preprocess_user_sentence(sentence, seqLen, eng_contractions, word_to_idx_dict, tk_test)
# tokens_in_numbers = get_tokens_in_numbers(tokens, seqLen)
# type_scores = trained_model.predict(tokens_in_numbers).tolist()

# # 每个sample sentence会得到四个score，分别对应'happiness', 'fun', 'sadness', 'hate'
# 选择取最高的score，如果最高score大于某个阈值（例如0.7），就判断为该类，如果最高的score达不到阈值，判断为没有特殊感情的neutral类
# print(type_scores)

### **Option B: Load model from a checkpoint**

In [ ]:
def create_lstm_model(seqLen, num_classes, vocab_size):
    input_tensor = Input(shape=(seqLen,), dtype='int32')
    mask = Masking(mask_value=0, input_shape=(seqLen, 50))(input_tensor)
    x = Embedding(vocab_size, 50, input_length=seqLen, trainable=False)(mask)
    x = LSTM(128, return_sequences=True)(x)
    x = LSTM(64, return_sequences=False)(x)
    x = Dense(16, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = Model(input_tensor, output_tensor)
    model.compile(optimizer=Adam(learning_rate=3e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    return model

In [ ]:
# Load model from checkpoint dir
checkpoint_dir = os.path.join(root_path, 'training_1')
latest_cp = tf.train.latest_checkpoint(checkpoint_dir)
# Create a new model instance
trained_model = create_lstm_model(seqLen, num_classes, vocab_size)

# Load the previously saved weights
trained_model.load_weights(latest_cp)

In [ ]:
# # Sample sentence
# sentence = 'I feel bad today. I think I should go to see a doctor. @father'
# # Test
# tk_test = TweetTokenizer()
# tokens = preprocess_user_sentence(sentence, seqLen, eng_contractions, word_to_idx_dict, tk_test)
# # tokens_in_numbers = get_tokens_in_numbers(tokens, seqLen)
# # type_scores = trained_model.predict(tokens_in_numbers).tolist()
# type_scores = trained_model.predict(tokens).tolist()

# # 每个sample sentence会得到四个score，分别对应'happiness', 'fun', 'sadness', 'hate'
# # 选择取最高的score，如果最高score大于某个阈值（例如0.7），就判断为该类，如果最高的score达不到阈值，判断为没有特殊感情的neutral类
# print(type_scores)

### **Make predictions based on user tweets**

In [ ]:
input_df = pd.read_csv(os.path.join(root_path, 'data/neighbor.csv'), 
                       encoding = 'unicode_escape',
                       dtype={'id': str},
                       na_filter=False)
input_df.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)
id_df = input_df[['id', 'center']]
inputs = input_df.drop(['center'], axis=1)
user_tweet_df = tweet_preprocess(inputs=inputs, minLen=minLen, tokenizer=tokenizer,sentence_tokenize=False)

In [ ]:
user_tweet_df.head(10)

In [ ]:
sentiment_df, partial_id_to_sentiment_dict = tweet_sentiment_predict(user_tweet_df, threshold=neutral_threshold, trained_model=trained_model, save_file=True)
sentiment_predicted_results, id_to_sentiment_dict = postprocessing(id_df, sentiment_df, partial_id_to_sentiment_dict)

In [ ]:
print(id_to_sentiment_dict)

In [ ]:
sentiment_predicted_results.head(10)

In [ ]:
# Save id -> sentiment labels
with open(os.path.join(root_path, 'data/neightbor_sentiments.csv'), 'w') as f:
    s = json.dumps(id_to_sentiment_dict)
    f.write(s)
f.close()